In [ ]:
import osmnx as ox
import networkx as nx
import pandas as pd
import os
import ollama
import json
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
import matplotlib.pyplot as plt # Needed for plotting the route

# --- Load all necessary data (ensure these are loaded for the full workflow) ---
data_folder = ''

# Load Road Network Graph
graph_file_path = os.path.join(data_folder, "road_network_Delhi_India.graphml")
G = ox.load_graphml(filepath=graph_file_path)
print("Road network graph loaded for full workflow.")

# Load Aligned EV Stations Data
aligned_ev_file_path = os.path.join(data_folder, 'aligned_ev_stations.csv')
df_stations = pd.read_csv(aligned_ev_file_path)
df_stations['nearest_node'] = df_stations['nearest_node'].astype(int)
print("Aligned EV stations data loaded for full workflow.")

# Initialize Geocoder - IMPORTANT: Use a unique user_agent string
geolocator = Nominatim(user_agent="tata_ev_nav_hackathon_2025_app")


# --- Environmental Constants (from Step 7) ---
CO2_EMISSION_FACTOR_PETROL_PER_LITER = 2.31 # kg CO2e/liter
GRID_EMISSION_FACTOR_INDIA = 0.8 # kg CO2e/kWh
CO2_ABSORPTION_PER_TREE_PER_YEAR = 22 # kg CO2/tree/year


# --- Vehicle Efficiency Data (from Step 7) ---
EV_EFFICIENCY_KM_PER_KWH = 6.5 # km per kWh (average efficiency for a compact EV in India)
ICE_EFFICIENCY_KM_PER_LITER = 15.0 # km per liter


# --- Core Routing Functions (from Step 6) ---
def get_nearest_node_to_point(graph, lat, lon):
    return ox.distance.nearest_nodes(graph, lon, lat)

def calculate_shortest_path(graph, origin_node, destination_node):
    return nx.shortest_path(graph, origin_node, destination_node, weight='length')

def find_charging_stations_along_route(route_nodes, df_ev_stations):
    route_nodes_set = set(route_nodes)
    stations_on_path_nodes = df_ev_stations[df_ev_stations['nearest_node'].isin(route_nodes_set)].copy()

    route_segment_lengths = {}
    cumulative_length = 0
    for i in range(len(route_nodes) - 1):
        u = route_nodes[i]
        v = route_nodes[i+1]
        edge_attributes = G.get_edge_data(u, v)
        current_segment_length = 0
        if edge_attributes:
            min_len_for_segment = float('inf')
            for k in edge_attributes:
                if 'length' in edge_attributes[k]:
                    min_len_for_segment = min(min_len_for_segment, edge_attributes[k]['length'])
            if min_len_for_segment != float('inf'):
                current_segment_length = min_len_for_segment
        cumulative_length += current_segment_length
        route_segment_lengths[v] = cumulative_length
    route_segment_lengths[route_nodes[0]] = 0

    stations_on_path_nodes['distance_from_route_origin'] = stations_on_path_nodes['nearest_node'].map(route_segment_lengths)
    stations_on_path_nodes.sort_values(by='distance_from_route_origin', inplace=True)

    return stations_on_path_nodes

# --- Green Impact Calculator Function (from Step 7) ---
def calculate_green_impact(distance_km, ev_efficiency_km_per_kwh, ice_efficiency_km_per_liter):
    if distance_km <= 0:
        return {
            "co2_emitted_ice_kg": 0, "co2_emitted_ev_kg": 0,
            "co2_saved_kg": 0, "liters_fuel_saved": 0,
            "equivalent_trees_planted": 0
        }
    fuel_consumed_ice_liters = distance_km / ice_efficiency_km_per_liter
    co2_emitted_ice = fuel_consumed_ice_liters * CO2_EMISSION_FACTOR_PETROL_PER_LITER
    energy_consumed_ev_kwh = distance_km / ev_efficiency_km_per_kwh
    co2_emitted_ev = energy_consumed_ev_kwh * GRID_EMISSION_FACTOR_INDIA
    co2_saved = co2_emitted_ice - co2_emitted_ev
    liters_fuel_saved = co2_saved / CO2_EMISSION_FACTOR_PETROL_PER_LITER
    equivalent_trees = co2_saved / CO2_ABSORPTION_PER_TREE_PER_YEAR
    return {
        "co2_emitted_ice_kg": round(co2_emitted_ice, 2),
        "co2_emitted_ev_kg": round(co2_emitted_ev, 2),
        "co2_saved_kg": round(co2_saved, 2),
        "liters_fuel_saved": round(liters_fuel_saved, 2),
        "equivalent_trees_planted": round(equivalent_trees, 2)
    }

# --- LLM Interaction Function for NLU (from Step 9) ---
def get_trip_parameters_from_llm(user_query, model_name='llama3.2'): # Use your working model name
    prompt = f"""
    You are an AI assistant that helps extract trip information from user queries for an EV journey planner.
    Your task is to identify the ORIGIN, DESTINATION, and optionally the EV_MODEL.
    If an EV_MODEL is not mentioned, use "Tata Nexon EV" as the default.
    If a specific origin or destination is mentioned, identify it precisely. If it's a general query (like "Find stations near X"), set both origin and destination to X.
    If you cannot identify a clear origin or destination, respond with "incomplete_query".

    Respond ONLY with a JSON object. Do NOT include any other text or markdown outside the JSON.

    Example 1:
    User: "Plan a trip from Connaught Place to Noida Sec 62 for my Tata Tiago EV."
    JSON: {{"origin": "Connaught Place, Delhi", "destination": "Noida Sec 62", "ev_model": "Tata Tiago EV", "intent": "plan_trip"}}

    Example 2:
    User: "I want to travel from Mumbai to Pune."
    JSON: {{"origin": "Mumbai", "destination": "Pune", "ev_model": "Tata Nexon EV", "intent": "plan_trip"}}

    Example 3:
    User: "Find charging stations near Connaught Place."
    JSON: {{"origin": "Connaught Place, Delhi", "destination": "Connaught Place, Delhi", "ev_model": "Tata Nexon EV", "intent": "find_stations"}}

    Example 4:
    User: "What's the best route?"
    JSON: {{"origin": null, "destination": null, "ev_model": null, "error": "incomplete_query"}}

    User Query: "{user_query}"
    JSON:
    """
    try:
        response = ollama.chat(model=model_name, messages=[
            {'role': 'system', 'content': prompt},
            {'role': 'user', 'content': user_query}
        ], options={'temperature': 0.1})
        llm_response_content = response['message']['content'].strip()
        # print(f"LLM Raw Response for '{user_query}': {llm_response_content}") # Uncomment for debugging
        parsed_json = json.loads(llm_response_content)
        return parsed_json
    except json.JSONDecodeError as e:
        print(f"Error parsing LLM response for '{user_query}' as JSON: {e}")
        print(f"Problematic content: {llm_response_content}")
        return {"error": "LLM response not valid JSON."}
    except Exception as e:
        print(f"An unexpected error occurred during LLM interaction for '{user_query}': {e}")
        return {"error": f"LLM interaction failed: {e}"}

# --- New Function: Geocoding Location Strings ---
def geocode_location(location_str):
    """Converts a location string (e.g., "Delhi") to (latitude, longitude)."""
    if not location_str:
        return None, None
    try:
        location = geolocator.geocode(location_str)
        if location:
            print(f"Geocoded '{location_str}' to: ({location.latitude}, {location.longitude})")
            return location.latitude, location.longitude
        else:
            print(f"Could not geocode '{location_str}'.")
            return None, None
    except Exception as e:
        print(f"Error geocoding '{location_str}': {e}")
        return None, None

# --- New Function: Orchestrate Full Trip Planning (with Debugging Prints) ---
def plan_ev_trip(user_query, df_stations, G):
    print(f"\n--- Processing Query: '{user_query}' ---")
    llm_params = get_trip_parameters_from_llm(user_query)
    print(f"Debug: LLM Parsed Params: {llm_params}")

    if "error" in llm_params:
        if llm_params.get("error") == "incomplete_query":
            return "I'm sorry, I couldn't understand your origin or destination. Please provide more details."
        return f"An error occurred during query processing: {llm_params['error']}"

    origin_str = llm_params.get('origin')
    destination_str = llm_params.get('destination')
    ev_model = llm_params.get('ev_model', 'Tata Nexon EV')
    intent = llm_params.get('intent', 'plan_trip')

    print(f"Debug: Origin Str: {origin_str}, Destination Str: {destination_str}, EV Model: {ev_model}, Intent: {intent}")

    if not origin_str or not destination_str:
        return "I need both an origin and a destination to plan a trip. Please try again."

    # Geocode origin and destination
    print(f"Debug: Geocoding '{origin_str}'...")
    origin_lat, origin_lon = geocode_location(origin_str)
    print(f"Debug: Geocoding '{destination_str}'...")
    destination_lat, destination_lon = geocode_location(destination_str)

    if origin_lat is None or destination_lat is None:
        return "I couldn't find precise coordinates for your origin or destination. Please try more specific locations within Delhi/NCR."

    # Find nearest graph nodes
    print(f"Debug: Finding nearest nodes for ({origin_lat}, {origin_lon}) and ({destination_lat}, {destination_lon})...")
    origin_node = get_nearest_node_to_point(G, origin_lat, origin_lon)
    destination_node = get_nearest_node_to_point(G, destination_lat, destination_lon)
    print(f"Debug: Nearest Origin Node: {origin_node}, Nearest Destination Node: {destination_node}")

    if origin_node is None or destination_node is None:
         return "Could not find a valid road network starting or ending point near your specified locations. Please try again with different points."


    # Handle 'find_stations' intent if detected
    if intent == 'find_stations':
        print(f"Debug: Handling 'find_stations' intent for '{origin_str}'...")
        stations_in_city = df_stations[(df_stations['city'].str.contains(origin_str.split(',')[0], case=False, na=False)) |
                                       (df_stations['address'].str.contains(origin_str.split(',')[0], case=False, na=False))]
        if not stations_in_city.empty:
            response_str = f"Here are some charging stations near '{origin_str}':\n"
            for idx, station in stations_in_city.head(5).iterrows():
                response_str += f"- {station['station_name']} ({station['address']}), Type: {station['charger_type_details']}\n"
            response_str += "\nFor a full list or to see them on a map, please check the app."
            return response_str
        else:
            return f"I couldn't find any charging stations near '{origin_str}' in our database."


    # Calculate shortest path
    print("Debug: Calculating shortest path...")
    try:
        route = calculate_shortest_path(G, origin_node, destination_node)
        # Manual route length calculation (bypassing osmnx.utils_graph)
        route_length_meters = 0
        for i in range(len(route) - 1):
            u = route[i]
            v = route[i+1]
            edge_data_dict = G.get_edge_data(u, v)
            if edge_data_dict:
                min_edge_length_segment = float('inf')
                for edge_id in edge_data_dict:
                    if 'length' in edge_data_dict[edge_id]:
                        min_edge_length_segment = min(min_edge_length_segment, edge_data_dict[edge_id]['length'])
                if min_edge_length_segment != float('inf'):
                    route_length_meters += min_edge_length_segment
        route_length_km = route_length_meters / 1000.0 # Convert meters to kilometers

        print(f"Debug: Route calculated. Length: {route_length_km:.2f} km.")

    except nx.NetworkXNoPath:
        return "I couldn't find a drivable path between these locations. They might be disconnected or too far apart in the road network."
    except Exception as e: # Catch any other routing errors
        return f"An error occurred while calculating the route: {e}"

    # Find charging stations along route
    print("Debug: Finding charging stations along route...")
    charging_stops = find_charging_stations_along_route(route, df_stations)
    print(f"Debug: Found {len(charging_stops)} charging stops on route.")

    # Calculate Green Impact
    print("Debug: Calculating Green Impact...")
    impact_results = calculate_green_impact(route_length_km, EV_EFFICIENCY_KM_PER_KWH, ICE_EFFICIENCY_KM_PER_LITER)
    print(f"Debug: Green Impact results: {impact_results}")

    # --- Natural Language Generation (NLG) using LLM ---
    print("Debug: Generating natural language summary using Python f-strings (bypassing LLM for NLG)...")
    summary_response = f"""
    Great trip planned from {origin_str} to {destination_str}!
    Here's your EV journey summary:
        Estimated Distance: {route_length_km:.2f} km
        Charging Stops Found: {len(charging_stops)} (on-route stations)
        First Potential Stop: {charging_stops.iloc[0]['station_name'] if not charging_stops.empty else 'N/A'}
        Assumed EV Model: {ev_model}
    Your EV contributes to a greener future!
        Estimated CO2 Saved: {impact_results['co2_saved_kg']:.2f} kg CO2
        Equivalent Liters of Petrol Saved: {impact_results['liters_fuel_saved']:.2f} liters
        Equivalent Trees Planted (annually): {impact_results['equivalent_trees_planted']:.2f} trees
    Enjoy your clean, efficient, and sustainable journey!
    """
    print("Debug: Hardcoded NLG response generated.")
    return summary_response.strip()
    
# --- Interactive Loop ---
print("\n--- EV Journey Planner: Interactive Mode (Type 'exit' to quit) ---")
while True:
    user_input = input("How can I help you plan your EV journey? ")
    if user_input.lower() == 'exit':
        print("Exiting journey planner. Goodbye!")
        break

    response = plan_ev_trip(user_input, df_stations, G)
    print(f"\nAI: {response}\n")

print("\n--- Full Workflow Orchestration complete. ---")

Road network graph loaded for full workflow.
Aligned EV stations data loaded for full workflow.

--- EV Journey Planner: Interactive Mode (Type 'exit' to quit) ---


How can I help you plan your EV journey?  Plan a trip from Connaught Place, Delhi to Sector 62, Noida for my Tata Punch EV.



--- Processing Query: 'Plan a trip from Connaught Place, Delhi to Sector 62, Noida for my Tata Punch EV.' ---
Debug: LLM Parsed Params: {'origin': 'Connaught Place, Delhi', 'destination': 'Sector 62, Noida', 'ev_model': 'Tata Punch EV', 'intent': 'plan_trip'}
Debug: Origin Str: Connaught Place, Delhi, Destination Str: Sector 62, Noida, EV Model: Tata Punch EV, Intent: plan_trip
Debug: Geocoding 'Connaught Place, Delhi'...
Geocoded 'Connaught Place, Delhi' to: (28.6314022, 77.2193791)
Debug: Geocoding 'Sector 62, Noida'...
Geocoded 'Sector 62, Noida' to: (28.6211447, 77.3643493)
Debug: Finding nearest nodes for (28.6314022, 77.2193791) and (28.6211447, 77.3643493)...
Debug: Nearest Origin Node: 1191294765, Nearest Destination Node: 9857973351
Debug: Calculating shortest path...
Debug: Route calculated. Length: 14.72 km.
Debug: Finding charging stations along route...
Debug: Found 8 charging stops on route.
Debug: Calculating Green Impact...
Debug: Green Impact results: {'co2_emitted_ic